In [1]:
import sys
import numpy as np
import pandas as pd

In [2]:
sys.path.append('/afs/cern.ch/eng/tracking-tools/modules')
import pymask as pm

In [3]:
command_log_file='log.madx'
stdout_file='stdout.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad = Madx(stdout=myFile,command_log=command_log_file)

# 1. Load thick optics

In [9]:
mad.input('''
Option,  warn,info;
System,"rm -rf temp";
System,"mkdir temp";

system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

call,file="lhc/lhc.seq";
call,file="slhc/toolkit/macro.madx"; 
call,file="slhc/hllhc_sequence.madx"; 
!call,file="slhc/ramp/opt_ramp_1000_1500_thin.madx";
call,file="slhc/ramp/opt_ramp_1000_1500.madx";

!exec,myslice;

exec,mk_beam(7000);
exec,check_ip(b1); exec,check_ip(b2); 
!call,file="slhc/toolkit/rematch_hllhc.madx"; 

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="initial_twiss.tfs";

''')

twiss_initial = mad.get_twiss_df(table_name='twiss')
summ_initial  = mad.table.summ.dframe()

# 2. Disable MS.10 in thick optics & rematch

In [10]:
mad.input('''
exec, DISABLE_SEXT(MS.10);
ksf=0.06;ksd=-0.099;
exec,set_sext_all(ksf,ksd,ksf,ksd); k2max=0.38;

if (match_w2==0){
call,file="slhc/toolkit/rematch_w.madx";
call,file="slhc/toolkit/rematch_disp.madx";
};
if (match_w2==2){
call,file="slhc/toolkit/rematch_w2.madx";
call,file="slhc/toolkit/rematch_disp2.madx";
};

call,file="slhc/toolkit/rematch_crabs.madx";
call,file="slhc/toolkit/mk_arc_trims.madx";

value,tarsqueeze, tar_xing_ir15, tar_xing_ir28, tar_on_disp;

exec,save_optics_hllhc(opt_ramp_woms10_1000_1500.madx);

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="last_twiss.tfs";

''')

twiss_final = mad.get_twiss_df(table_name='twiss')
summ_final  = mad.table.summ.dframe()

# 3. sanity checks

In [11]:
# beta in IP1,2,5,8
print('Betas:')
print(twiss_final.loc['ip1:1'].betx, twiss_final.loc['ip2:1'].betx, twiss_final.loc['ip5:1'].betx, twiss_final.loc['ip8:1'].betx)

# tunes & chromaticity
print('Initial tunes and chroma:')
print(summ_initial['q1'].values[0], summ_initial['q2'].values[0], summ_initial['dq1'].values[0], summ_initial['dq2'].values[0])
print('Final tunes and chroma::')
print(summ_final['q1'].values[0], summ_final['q2'].values[0], summ_final['dq1'].values[0], summ_final['dq2'].values[0])


# MS.10 k2l
print('Initial k2l MS.10:')
print(twiss_initial[twiss_initial.index.str.contains('ms.10')]['k2l'])
print('Final k2l MS.10:')
print(twiss_final[twiss_final.index.str.contains('ms.10')]['k2l'])


Betas:
1.0000000000006846 10.000000000022213 0.9999999999993258 1.4999999999720754
Initial tunes and chroma:
62.31000000000256 60.31999999999985 1.9991835529253876 2.000640087373995
Final tunes and chroma::
62.31000000000425 60.320000000002665 1.9990835440353294 2.0007425902690557
Initial k2l MS.10:

ms.10r1.b1:1   -0.082879
ms.10l5.b1:1    0.052796
ms.10r5.b1:1   -0.096072
ms.10l1.b1:1    0.069559
Name: k2l, dtype: float64
Final k2l MS.10:

ms.10l5.b1:1    0.0
ms.10r5.b1:1    0.0
ms.10l1.b1:1    0.0
ms.10r1.b1:1    0.0
Name: k2l, dtype: float64


# 4. Load thick and slice to make thin

In [12]:
mad.input('''
system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
call,file="slhc/toolkit/macro.madx"; 

call,file="opt_ramp_woms10_1000_1500.madx";

exec,myslice;
call,file="slhc/toolkit/rematch_hllhc.madx"; 

exec,save_optics_hllhc(opt_ramp_woMS10_1000_1500_thin.madx);
''')


True

# 5. Sanity check for thin optics

In [11]:
command_log_file='log2.madx'
stdout_file='stdout2.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad2 = Madx(stdout=myFile,command_log=command_log_file)

In [12]:
mad2.input('''
Option,  warn,info;
System,"rm -rf temp";
System,"mkdir temp";

system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

call,file="lhc/lhc.seq";
call,file="slhc/toolkit/macro.madx"; 
call,file="slhc/hllhc_sequence.madx"; 

exec, DISABLE_SEXT(MS.10);

call,file="opt_ramp_woms10_1000_1500_thin.madx";

exec,myslice;

exec,mk_beam(7000);
exec,check_ip(b1); exec,check_ip(b2); 
!call,file="slhc/toolkit/rematch_hllhc.madx"; 

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="initial_twiss.tfs";

''')

twiss_initial = mad2.get_twiss_df(table_name='twiss')
summ_initial  = mad2.table.summ.dframe()

In [13]:
# beta in IP1,2,5,8
print('Betas:')
print(twiss_initial.loc['ip1:1'].betx, twiss_initial.loc['ip2:1'].betx, twiss_initial.loc['ip5:1'].betx, twiss_initial.loc['ip8:1'].betx)

# tunes & chromaticity
print('Initial tunes and chroma:')
print(summ_initial['q1'].values[0], summ_initial['q2'].values[0], summ_initial['dq1'].values[0], summ_initial['dq2'].values[0])


# MS.10 k2l
print('Initial k2l MS.10:')
print(twiss_initial[twiss_initial.index.str.contains('ms.10')]['k2l'])


Betas:
0.9999999998790806 10.000000001197263 0.9999999998712145 1.4999999999444529
Initial tunes and chroma:
62.31000046549589 60.32000046103872 1.983767845104012 2.0003057770168198
Initial k2l MS.10:

ms.10r1.b1:1    0.0
ms.10l5.b1:1    0.0
ms.10r5.b1:1    0.0
ms.10l1.b1:1    0.0
Name: k2l, dtype: float64
